In [342]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt

In [343]:
df = pd.read_csv('titanic.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [344]:
df.tail()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.00,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.00,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q


In [345]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [346]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [347]:
df.isnull().sum()/df.shape[0]*100

PassengerId     0.000000
Survived        0.000000
Pclass          0.000000
Name            0.000000
Sex             0.000000
Age            19.865320
SibSp           0.000000
Parch           0.000000
Ticket          0.000000
Fare            0.000000
Cabin          77.104377
Embarked        0.224467
dtype: float64

In [348]:
#check duplicates
df.duplicated().sum()

np.int64(0)

In [349]:
# Löschen der Spalte 'Cabin' aus dem DataFrame weil 77% missing values
df.drop('Cabin', axis=1, inplace=True)
#Age abrunden
df['Age'] = np.floor(df['Age'])
#missing values treatment

df['Age'] = df['Age'].fillna(df['Age'].median())
df['Age'] = df['Age'].astype(int)
df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])



In [350]:
#check missing values
df.isnull().sum()/df.shape[0]*100

PassengerId    0.0
Survived       0.0
Pclass         0.0
Name           0.0
Sex            0.0
Age            0.0
SibSp          0.0
Parch          0.0
Ticket         0.0
Fare           0.0
Embarked       0.0
dtype: float64

In [351]:
#extrahiere new column 'Titel',das könnte wichtig für datenanalyse sein?
df["Title"] = df["Name"].apply(lambda x: x.split(",")[1].split(".")[0].strip())
df["Name"] = df["Name"].apply(lambda x: ' '.join([word for word in x.split() if not word.endswith('.')]))


In [352]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Title
0,1,0,3,"Braund, Owen Harris",male,22,1,0,A/5 21171,7.2500,S,Mr
1,2,1,1,"Cumings, John Bradley (Florence Briggs Thayer)",female,38,1,0,PC 17599,71.2833,C,Mrs
2,3,1,3,"Heikkinen, Laina",female,26,0,0,STON/O2. 3101282,7.9250,S,Miss
3,4,1,1,"Futrelle, Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,S,Mrs
4,5,0,3,"Allen, William Henry",male,35,0,0,373450,8.0500,S,Mr


In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler


X = df[['Pclass', 'Sex', 'Age', 'Fare', 'Embarked','SibSp','Parch','Title']]  # Features
X = pd.get_dummies(X, columns=['Embarked','Sex','Title'], drop_first=True)
y = df['Survived']  # Zielvariable

# KFold erstellen
k = 10
kf = KFold(n_splits=k, shuffle=True, random_state=42)


model1 = LogisticRegression(solver='lbfgs', max_iter=1000, C=0.1)
model2 = DecisionTreeClassifier(random_state=42)
model3 = KNeighborsClassifier(n_neighbors=3)

# K-Fold Cross-Validation
lreg_accuracies = []
dt_accuracies = []
k_nn_accuracies = []
#X = X.copy()  # warning
#X['Sex'] = X['Sex'].map({'male': 0, 'female': 1})
#X['Embarked'] = X['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})

for train_index, test_index in kf.split(X):
    # split data in test und training Data
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]
    
    #logistische Regression
    model1.fit(X_train, y_train)
    y_pred_log_reg = model1.predict(X_test)
    log_reg_accuracy = accuracy_score(y_test, y_pred_log_reg)
    lreg_accuracies.append(log_reg_accuracy)
    
    #Decision Tree
    model2.fit(X_train, y_train)
    y_pred_dt = model2.predict(X_test)
    dt_accuracy = accuracy_score(y_test, y_pred_dt)
    dt_accuracies.append(dt_accuracy)

    #K-nearest neighbor mit k=3
    model3.fit(X_train, y_train)
    y_pred_knn = model3.predict(X_test)
    k_nn_accuracy = accuracy_score(y_test, y_pred_knn)
    k_nn_accuracies.append(k_nn_accuracy)

# Durchschnitt berechnen
log_reg_avg_accuracy = sum(lreg_accuracies) / len(lreg_accuracies)
dt_avg_accuracy = sum(dt_accuracies) / len(dt_accuracies)
k_nn_avg_accuracy = sum(k_nn_accuracies) / len(k_nn_accuracies)

print(f"Logistic Regression: {log_reg_avg_accuracy:.4f}")
print(f"Decision Tree: {dt_avg_accuracy:.4f}")
print(f"K-nearest neighbor mit K=3: {k_nn_avg_accuracy:.4f}")





    

Logistic Regression: 0.8193
Decision Tree: 0.8024
K-nearest neighbor mit K=3: 0.7262
